In [1]:
#Import Libraries

from pyspark.sql import SparkSession
from pyspark.sql.functions import when

In [2]:
#Configuration Jobs

spark = SparkSession \
    .builder \
    .appName("Flights") \
    .getOrCreate()

In [4]:
#Load csv Data to DataFrame

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [7]:
#Flights
flightsDF= spark.read.option("header","true").csv("./Data/flights.csv")
#Airports
airportsDF= spark.read.option("header","true").csv("./Data/airports.csv")
#Airline
airlinesDF= spark.read.option("header","true").csv("./Data/airlines.csv")

In [10]:
airportsDF.show(2)
airlinesDF.show(2)
flightsDF.show(2)

+---------+--------------------+---------+-----+-------+--------+---------+
|IATA_CODE|             AIRPORT|     CITY|STATE|COUNTRY|LATITUDE|LONGITUDE|
+---------+--------------------+---------+-----+-------+--------+---------+
|      ABE|Lehigh Valley Int...|Allentown|   PA|    USA|40.65236|-75.44040|
|      ABI|Abilene Regional ...|  Abilene|   TX|    USA|32.41132|-99.68190|
+---------+--------------------+---------+-----+-------+--------+---------+
only showing top 2 rows

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
+---------+--------------------+
only showing top 2 rows

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+